# P-Hacking

Das Konzept P-Hacking bedeutet, dass man absichtlich so lange die Auswertung der eigenen Daten verändert, bis genau die Werte herauskommen, die zur eigenen Hypothese passen.
Dieses Verhalten ist ein Problem für alle Bereiche:
In der Wissenschaft werden unter Umständen falsche Hypothesen (fälschlicherweise) empirisch untermauert.
In der Praxis bedeutet es, dass ggf. defekte Modelle in den Betrieb aufgenommen werden.
Sobald sich jemand auf das falsche Modell verlässt, kann dies zu ernsthaften Schäden an Menschen oder Umwelt führen.
Ebenso sind finanzielle Schäden nicht ausgeschlossen.
Weiterführende Infos gibt es z. B. auf
[Wikipedia](https://de.wikipedia.org/wiki/P-Hacking).

In [ ]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

Zunächst wird wieder der Zufallsgenerator fixiert.

In [ ]:
np.random.seed(0)

Es werden wieder zufällige Attribute erzeugt.

In [ ]:
df = pd.DataFrame(np.random.randint(0, 10, size=(50, 26)), columns=list('ABCDEFGHIJKLMNOPQRSTUVWXYZ'))
df

Wie stark sind die Korrelationen ausgeprägt?
Dies wird nun visualisiert:

In [ ]:
fig = plt.figure(figsize=(19, 15))
df_corr = df.corr()
plt.matshow(df_corr, fignum=fig.number, cmap='RdBu', vmin=-1, vmax=1)
plt.xticks(range(df.shape[1]), df.columns)
plt.yticks(range(df.shape[1]), df.columns)
plt.colorbar()
plt.show()

Die Korrelationsmatrix ist symmetrisch, wenn A und B korrelieren, korrelieren auch B und A.#
Deswegen blenden wir nur die eine Hälfte der Matrix aus.

In [ ]:
upper_right_matrix = df_corr.where(np.triu(np.ones(df_corr.shape), k=1).astype(bool)) 

fig = plt.figure(figsize=(19, 15))
plt.matshow(upper_right_matrix, fignum=fig.number, cmap='RdBu', vmin=-1, vmax=1)
plt.xticks(range(upper_right_matrix.shape[1]), upper_right_matrix.columns)
plt.yticks(range(upper_right_matrix.shape[1]), upper_right_matrix.columns)
plt.colorbar()
plt.show()

Eine reduzierte Darstellung lässt sich wie folgt finden:

In [ ]:
stacked_df = upper_right_matrix.stack().reset_index()
stacked_df.columns = ["attribute_A", "attribute_B", "r"]
stacked_df

## Feature-Engineering falsch gemacht

Nun werden wir so lange neue Features erstellen, bis wir (mindestens) eines finden, dass mit einem Attribut korreliert.
Hierhinter steht keinerlei Theorie, es sind reine Zufallsdaten.
Dafür werden die Attribute addiert und multipliziert.
Andere Operatoren wie Division und Subtraktion wären natürlich genauso möglich.
Darüber hinaus könnte man noch viel mehr Funktionen verwenden, wie Sinus, Logarithmus etc.

In [ ]:
# Hier werden die Additionen zweier Attribute gespeichert
additions = []

# Hier werden die Multiplikationen zweier Attribute gespeichert
multiplications = []

for index, (column_A, column_B, r) in stacked_df.iterrows():

    addition = df[column_A] + df[column_B]
    addition.name = f"{column_A} + {column_B}"
    additions.append(addition)

    multiplication = df[column_A] * df[column_B]
    multiplication.name = f"{column_A} * {column_B}"
    multiplications.append(multiplication)


Diese kreierten Attribute fügen wir nun dem DataFrame hinzu.

In [ ]:
df = df.assign(**{series.name : series for series in additions})
df = df.assign(**{series.name : series for series in multiplications})

df

In [ ]:
fig = plt.figure(figsize=(19, 15))
plt.matshow(df.corr(), fignum=fig.number, cmap='RdBu', vmin=-1, vmax=1)
plt.colorbar()
plt.show()

In [ ]:
df_corr = df.corr()
upper_right_matrix = df_corr.where(np.triu(np.ones(df_corr.shape), k=1).astype(bool)) 
df_high_corr = upper_right_matrix[(df_corr > .5)]

Es sind sehr viele Attribute geworden und dies lässt sich nun auch schwer visuell inspizieren.
Deswegen untersuchen wir diese Korrelationskoeffizienten nun weiter.
Dafür werden diese zunächst in eine Liste überführt.
Dadurch geht die Information verloren, wie genau die Zeilen heißen, die korrelieren.

In [ ]:
cor_values = df_corr.values.flatten()
pd.DataFrame(cor_values, columns=["Korrelationskoeffizient"]).plot.hist()
plt.show()

Sehr viele Korrelationen sind nur sehr schwach ausgeprägt.
Deswegen kann man diese im Histogramm besser erkennen als die stärkeren Korrelationen.
Um die stärker ausgeprägten Korrelationen besser sehen zu können, entfernen wir den mittleren Teil.
Außerdem ist weniger von Interesse, ob eine Korrelation positiv oder negativ ist.
Die hier eingesetzte Funktion `abs()` nimmt den absoluten Betrag.

In [ ]:
pd.DataFrame(abs(cor_values[((cor_values < -.5) | (cor_values > .5))])).plot.hist()
plt.show()

Es sieht so aus, als ob die Suche erfolgreich war.
Bei 0,9 gibt es mehrere Einträge.
Hier zoomen wir weiter heran.

In [ ]:
pd.DataFrame(abs(cor_values[((cor_values < -.9) | (cor_values > .9))])).plot.hist()
plt.show()

Es sind auffällig viele hohe Korrelationen dabei.
Dies wird nun weiter inspiziert.

## Inspiziere die Korrelationen der Features

Zunächst filtern wir die Korrelationen aus der vorherigen Matrix.

In [ ]:
correlating = []

for attribute_A in df_high_corr.columns:
    for attribute_B, correlation_coefficient in df_high_corr[attribute_A].loc[
        ~pd.isnull(df_high_corr[attribute_A])
    ].iteritems():
        correlation_coefficient = df_high_corr[attribute_A].loc[attribute_B]
        correlating.append((attribute_A, attribute_B, correlation_coefficient))

df_corr_summary = pd.DataFrame(correlating, columns = ["Attribut A", "Attribut B", "r"])
df_corr_summary

Nun werden alle trivialen Fälle herausgefiltert.
Dazu gehören solche, wo in beiden Termen das gleiche Attribut vorkommt.

In [ ]:
interesting_indices = []

for index, attribute_A, attribute_B, r in df_corr_summary.itertuples():

    if attribute_A in attribute_B or attribute_B in attribute_A:
        # einfacher Fall: A ~ A*A oder A ~ A+A
        continue

    variables_A = [v for v in attribute_A if v not in (" ", "*", "+")]
    variables_B = [v for v in attribute_B if v not in (" ", "*", "+")]

    if set(variables_A).intersection(variables_B):
        # einfacher Fall: ein Operand kommt auf beiden Seiten vor
        continue

    interesting_indices.append(index)

df_corr_summary.loc[interesting_indices].plot.hist()
plt.show()

In [ ]:
df_corr_summary.loc[interesting_indices].sort_values(by="r", ascending=False)

Solche zufällig entstehenden Korrelationen treten dadurch auf, dass wir lang genug verschiedene Terme durchprobiert haben.
Diese Korrelationen würden mit einer anwachsenden Datenmenge (mehr Beobachtungen je Attribut) höchstwahrscheinlich verschwinden.
Häufig kann die Aufteilung in Trainings- und Testset helfen, diese Situation zu erkennen.
Allerdings gibt es immer eine gewisse Wahrscheinlichkeit, dass zufälligerweise das Testset so ausgewählt worden ist, dass es keinen Hinweis gibt, dass etwas im Argen liegt.

Ein Lernalgorithmus könnte auf Basis von solchen wahllos kreierten Features Korrelationen finden, die diese zu falsche Vorhersagen verleitet.
Deswegen ist es wichtig, dass Lernverfahren für die Größe des vorliegenden Datensatzes nie zu komplex werden.